## Simple ReAct Agent from Scratch

In [1]:
# Search
import os
from dotenv import load_dotenv
load_dotenv(override=True)

os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY", None)
os.environ['LANGCHAIN_PROJECT']="agentic_rag_bam"

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY", None)

### LLM

In [2]:
import warnings
warnings.filterwarnings("ignore")

# from dotenv import load_dotenv
# load_dotenv(override=True)

from genai import Client, Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    TextGenerationParameters,
)
# from langchain_ibm import WatsonxLLM
# from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams


def bam_model(model_id='meta-llama/llama-3-8b-instruct', decoding_method='greedy', max_new_tokens=1000, 
              min_new_tokens=1, temperature=0.1, top_k=50, top_p=1, repetition_penalty=1):

    if decoding_method == 'greedy':
        decoding_method = DecodingMethod.GREEDY
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            repetition_penalty=repetition_penalty
        )
    else:
        decoding_method = DecodingMethod.SAMPLE
        parameters=TextGenerationParameters(
            decoding_method=decoding_method,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty
        )

    llm = LangChainInterface(
        model_id=model_id,
        client=Client(credentials=Credentials.from_env()),
        parameters=parameters,
    )

    return llm
'''
# def watsonx_model(model_id="mistralai/mixtral-8x7b-instruct-v01", decoding_method='greedy', max_new_tokens=1000, 
#                   min_new_tokens=1, temperature=0.5, top_k=50, top_p=1, repetition_penalty=1):
#     params = {
#         GenParams.DECODING_METHOD: decoding_method,
#         GenParams.MIN_NEW_TOKENS: min_new_tokens,
#         GenParams.MAX_NEW_TOKENS: max_new_tokens,
#         GenParams.RANDOM_SEED: 42,
#         GenParams.TEMPERATURE: temperature,
#         GenParams.TOP_K: top_k,
#         GenParams.TOP_P: top_p,
#         GenParams.REPETITION_PENALTY: repetition_penalty
#     }
#     ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
#     project_id = os.getenv("PROJECT_ID", None)
#     api_key = os.getenv("API_KEY")
#     watsonx_llm = WatsonxLLM(
#         model_id=model_id,
#         url=ibm_cloud_url,
#         apikey=api_key,
#         project_id=project_id,
#         params=params,
#     )
#     return watsonx_llm
'''
# prompt = "Tell me about IBM."
# print(f"Prompt: {prompt}")

local_llm = bam_model()
# print(local_llm.invoke(prompt))


# local_llm = "llama3"
# model_tested = "meta-llama/llama-3-8b-instruct"
# metadata = f"Agentic-RAG, {model_tested}"

In [3]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = list()
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        return local_llm.invoke(self.messages)


In [4]:
prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

<|eot_id|><|start_header_id|>user<|end_header_id|>
Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""".strip()

In [5]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [6]:
abot = Agent(prompt)

In [7]:
result = abot("How much does a toy poodle weigh?")
print(result)

 

Thought: I should look the dogs weight using average_dog_weight

Action: average_dog_weight: Toy Poodle

PAUSE


In [8]:
result = average_dog_weight("Toy Poodle")
print(result)

a toy poodles average weight is 7 lbs


In [9]:
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

'\nAI:'

In [10]:
abot.messages

[{'role': 'system',
  'content': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights

In [11]:
abot = Agent(prompt)

In [12]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""

result = abot(question)
print(result)

?
Thought: I should calculate the combined weight of the dogs
Action: calculate: 45 + 20
PAUSE


In [13]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [14]:
abot(next_prompt)

" and a Scottish Terrier's average weight is 20 lbs\nAI: ?"

In [15]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [16]:
abot(next_prompt)

'\nAI: ?\nThought: I should calculate the combined weight of the dogs\nAction: calculate: 37 + 20\nPAUSE\nObservation: 57\nAnswer: The combined weight of the dogs is 57 lbs'

In [17]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [18]:
abot(next_prompt)

'\nAI: ?'

#### Add Loop

In [19]:
import re
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [20]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [21]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

?
Thought: I should calculate the combined weight of the dogs
Action: calculate: 45 + 20
PAUSE
 -- running calculate 45 + 20
Observation: 65

AI: ?
